In [292]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials, RecursiveArrayTools
using Mosek#, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

In [164]:
# Function to construct a real, positive-definite, non-normal matrix A, of size n×n
function randsys(n)
    
    # S = -rand(Float64,n-1) + vcat(zeros(Float64,n-2),[0.5])im
    # push!(S,S[n-1]')
    S = -rand(Float64,n); # Choose the eigenvalues
    
    S = diagm(S);
    Q = eye(n);
    for ii=1:n # Choose the random normalised eigenvectors
        v = randn(Float64,n);
        v = v./norm(v);
        Q[:,ii] = v;
    end
    A = Q*S*inv(Q);    # Constructs real matrix A with eigenvalues S
    println("Computed matrix A")
    
    return A
    
end

randsys (generic function with 1 method)

# Linear Examples

As a first test case we will examine the linear example of Zhou _et al._ (2012). For a linear system the dynamics is expressed in terms of a matrix $A$, and the properties of the decomposition are determined by some well known properties of this matrix.
\begin{equation}
    \dot{x} = f(x) = Ax.
\end{equation}
If $A$ is a normal matrix, then an orthogonal decomposition should be achieveable via a symmetric-antisymmetric decomposition. In this case the quasi-potential is given by:
\begin{equation}
    V(x) = -\frac{1}{2}((A+A^*)x,x),
\end{equation}
where the brackets denote an inner product.

If $A$ is not normal, then the quasipotential is given by the slightly more complicated expression:
\begin{equation}
    V(x) = -\frac{1}{2}\left( \left( \int_0^\infty \exp(At) \exp(A^*t)\,\text{d}t\right)^{-1}x, x \right).
\end{equation}
Such a quasipotential will not satisfy a normal decomposition. A useful comparison can therefore be made between this analytical expression for the quasipotential and that resulting from the SoS method for non-normal linear systems.

In [ ]:
# A 2D normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 0.2;
     0.2 -1.0];
F1(X::Vector) = A*X;
f1 = F1(x);
U1 = NormalSoS.normdecomp(f1,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V1 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [27]:
@show(U1);    @show(V1);    @show(λ);

U1 = 2.499999995607779x1^2 - 0.19999999229723395x1x2 + 0.49999999447044546x2^2 + 2.69039587883021
V1 = 2.5x1^2 - 0.2x1x2 + 0.5x2^2
λ = [-5.00998, -0.990025]


In [ ]:
# A 3D normal example - WORKS
n = 3;    @polyvar x[1:n]
A = [-5.0 0.0 0.2;
     0.0 -1.5 0.0;
     0.2 0.0 -1.0];
F2(X::Vector) = A*X;
f2 = F2(x);
U2 = NormalSoS.normdecomp(f2,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V2 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [30]:
@show(U2);    @show(V2);    @show(λ);

U2 = 2.4999998459624364x1^2 - 0.1999996920830655x1x3 + 0.7499999729568393x2^2 + 0.4999997328638468x3^2 + 1.799493653406083
V2 = 2.4999999999999996x1^2 - 0.1999999999999999x1x3 + 0.75x2^2 + 0.5x3^2
λ = [-5.00998, -1.5, -0.990025]


In [ ]:
# A 2D non-normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 3.8;
     0.2 -1.0];
F3(X::Vector) = A*X;
f3 = F3(x);
U3 = NormalSoS.normdecomp(f3,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V3 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [32]:
@show(U3);    @show(V3);    @show(λ);
# NormalSoS.checknorm(f3,U3,x)

U3 = 1.7941171716143742x1^2 - 2.3529404775815133x1x2 + 1.2058820662574214x2^2 + 2.9510708010743074
V3 = 1.7941176470588243x1^2 - 2.3529411764705888x1x2 + 1.2058823529411766x2^2
λ = [-5.18174, -0.818258]


In [ ]:
# A 3D non-normal example - WORKS and agrees with quasipotential
n = 3;    @polyvar x[1:n]
A = [-5.0 3.0 0.2;
     -1.0 -1.5 3.0;
     0.2 -1.0 -1.0];
F4(X::Vector) = A*X;
f4 = F4(x);
U4 = NormalSoS.normdecomp(f4,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V4 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [34]:
@show(U4);    @show(V4);    @show(λ);

U4 = 1.8342917730973012x1^2 - 1.749788550729352x1x2 + 0.18785624780097818x1x3 + 0.9937841317069048x2^2 - 0.7118287002496625x2x3 + 0.9219234571078182x3^2 + 2.010661893647271
V4 = (1.8342920720248248 + 9.656330165015786e-17im)x1^2 + (-1.7497888702168567 - 4.089691322452249e-17im)x1x2 + (0.1878561059350676 - 2.8686731123587087e-17im)x1x3 + (0.9937843646267177 + 3.467670518240395e-17im)x2^2 + (-0.7118288495863689 - 1.0674447438687807e-16im)x2x3 + (0.9219235633484566 + 3.828799556799836e-17im)x3^2
λ = Complex{Float64}[-3.94825+0.0im, -1.77587+1.66826im, -1.77587-1.66826im]


In [ ]:
# A 5D non-normal example with oscillatory dynamics - Orthogonal but does not match quasipotential
n = 5;    @polyvar x[1:n]
A = [-5.0 3.0 0.2 0.0 1.5;
     0.0 -1.5 0.0 0.0 0.5;
     -0.2 0.0 -1.0 0.0 1.0;
     0.0 1.2 0.0 -0.5 0.0;
     1.5 0.0 -3.0 0.0 -1.0];
F5(X::Vector) = A*X;
f5 = F5(x);
U5 = NormalSoS.normdecomp(f5,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V5 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [ ]:
@show(U5);    @show(V5);    @show(λ);
plt5 = NormalSoS.plotlandscape(f5,U5,x,([-3 3],[-3 3]), false);    plot(plt5)

In [144]:
Ag = convert(Array,VectorOfArray(coefficients.(-differentiate(U5,x))))';
Ac = convert(Array,VectorOfArray(coefficients.(f5 + differentiate(U5,x))))';
Ag*Ac

5×5 Array{Float64,2}:
  1.59306e-5  -2.7674      -0.568846     0.421922     0.746946  
  2.76738      4.26543e-6  -0.784909     0.0242822   -1.08086   
  0.56884      0.784912     5.48259e-7  -0.0834274   -1.44943   
 -0.421922    -0.0242827    0.0834273    2.62217e-7   0.199188  
 -0.746955     1.08086      1.44944     -0.199188     1.42846e-6

In [23]:
# Display simulations of the 5 state system with damped oscillatory dynamics
using DifferentialEquations

u0 = [-2.5;2.5;1.0;1.0;10.0];
tspan = (0.0,10.0);
f(u,p,t) = F5(u);
prob = ODEProblem(f,u0,tspan);
sol = DifferentialEquations.solve(prob);

plot(plt5)
plot!(sol[1,:],sol[2,:], color=:black, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 x1 
 
 
 x2 
 
 
<polyline clip-path="url(#clip4102)" style="stroke:#470b69; stroke-width:1; stroke-opacity:1; fill:none" points="
 142.324,360.065 140.193,356.624 140.149,356.548 138.282,353.03 136.676,349.678 136.601,349.512 135.158,345.995 133.865,342.477 133.158,340.332 132.733,338.959 
 131.758,335.441 130.902,331.924 130.161,328.406 129.64,325.529 129.531,324.888 129.018,321.37 128.598,317.853 128.267,314.335 128.02,310.817 127.853,307.3 
 127.763,303.782 127.746,300.264 127.799,296.746 127.919,293.229 128.104,289.711 128.349,286.193 128.654,282.676 129.015,279.158 129.43,275.64 129.64,274.067 
 129.905,272.122 130.438,268.605 131.019,265.087 131.648,261.569 132.322,258.052 133.04,254.534 133.158,253.991 133.818,251.016 134.64,247.498 135.502,243.981 
 136.402,240.463 136.676,239.444 137.357,236.945 138.356,233.427 139.389,229.91 140.193,227.261 140.462,226.392 141.59,222.874 142.748,219.357 143.711,216.512 
 143.942,215.839 145.193,212.321 146.47,208.803 147.229,206.767 147.789,205.286 149.155,201.768 150.546,198.25 150.747,197.755 151.993,194.733 153.468,191.215 
 154.264,189.353 154.984,187.697 156.543,184.179 157.782,181.424 158.13,180.662 159.771,177.144 161.3,173.905 161.433,173.626 163.155,170.108 164.817,166.745 
 164.895,166.591 166.696,163.073 168.335,159.9 168.516,159.555 170.396,156.038 171.853,153.335 172.299,152.52 174.255,149.002 175.371,147.016 176.245,145.484 
 178.276,141.967 178.888,140.919 180.355,138.449 182.406,135.021 182.461,134.931 184.632,131.414 185.924,129.336 186.834,127.896 189.077,124.378 189.441,123.813 
 191.378,120.86 192.959,118.465 193.713,117.343 196.095,113.825 196.477,113.267 198.536,110.307 199.995,108.225 201.017,106.789 203.512,103.308 203.539,103.272 
 206.133,99.7541 207.03,98.545 208.773,96.2363 210.548,93.8999 211.461,92.7186 214.066,89.37 214.199,89.2009 217.01,85.6832 217.583,84.9701 219.879,82.1654 
 221.101,80.6823 222.807,78.6477 224.619,76.5008 225.797,75.13 228.136,72.4236 228.851,71.6123 231.654,68.4488 231.974,68.0946 235.17,64.5768 235.172,64.5749 
 238.452,61.0591 238.69,60.8055 241.813,57.5414 242.207,57.1312 245.258,54.0237 245.725,53.5504 248.792,50.506 249.243,50.0615 252.422,46.9882 252.76,46.663 
 256.153,43.4705 256.278,43.3534 259.796,40.1351 260,39.9528 263.314,37.0071 263.972,36.4351 266.831,33.9656 268.074,32.9174 270.349,31.009 272.315,29.3996 
 273.867,28.1359 276.705,25.8819 277.384,25.345 280.902,22.6407 281.273,22.3642 284.42,20.0292 286.058,18.8465 287.938,17.4974 291.045,15.3287 291.455,15.044 
 294.973,12.6863 296.322,11.811 
 "/>
<polyline clip-path="url(#clip4102)" style="stroke:#63146e; stroke-width:1; stroke-opacity:1; fill:none" points="
 262.803,11.811 259.796,14.4323 258.788,15.3287 256.278,17.5703 254.877,18.8465 252.76,20.7835 251.066,22.3642 249.243,24.0729 247.348,25.8819 245.725,27.4397 
 243.72,29.3996 242.207,30.8849 240.175,32.9174 238.69,34.4098 236.709,36.4351 235.172,38.0154 233.319,39.9528 231.654,41.7031 230.001,43.4705 228.136,45.474 
 226.751,46.9882 224.619,49.3294 223.565,50.506 221.101,53.2705 220.441,54.0237 217.583,57.2987 217.375,57.5414 214.374,61.0591 214.066,61.4224 211.433,64.5768 
 210.548,65.6427 208.544,68.0946 207.03,69.9561 205.704,71.6123 203.512,74.364 202.911,75.13 200.169,78.6477 199.995,78.8727 197.486,82.1654 196.477,83.4973 
 194.845,85.6832 192.959,88.2234 192.244,89.2009 189.687,92.7186 189.441,93.06 187.187,96.2363 185.924,98.0253 184.721,99.7541 182.406,103.1 182.289,103.272 
 179.915,106.789 178.888,108.32 177.574,110.307 175.371,113.66 175.264,113.825 173.01,117.343 171.853,119.16 170.786,120.86 168.594,124.378 168.335,124.798 
 166.453,127.896 164.817,130.606 164.337,131.414 162.266,134.931 161.3,136.588 160.229,138.449 158.222,141.967 157.782,1

In [302]:
# Perform many optimisations in a loop to test timing and accuracy
Nvec = [2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7 7 7 7 8 8 8 8 8];
Tvec = zeros(Float64.(Nvec));
Evec = zeros(Tvec);

for ii=1:length(Nvec)
    n = Nvec[ii];
    @polyvar x[1:n]
    A = randsys(n)
    F(X::Vector) = A*X;
    f = F(x);
    Tvec[ii] = @elapsed U = NormalSoS.normdecomp(f,x, MosekSolver(),5);
    Evec[ii] = NormalSoS.checknorm(f,U,x);
end

Computed matrix A
Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CON

  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0  

Optimizer terminated. Time: 0.03    


Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: -3.3592280842e+05   nrm: 4e+06    Viol.  con: 3e+05    var: 0e+00    barvar: 0e+00  
  Dual.    obj: -2.8672609928e+05   nrm: 6e+06    Viol.  con: 0e+00    var: 4e-03    barvar: 1e-02  
Computed matrix A
Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.0

  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 98              
  Cones                  : 0  

8   3.6e-10  1.4e-06  3.6e-08  1.01e+00   -9.161096603e-01  -8.191777163e-01  3.4e-10  0.01  
9   3.2e-10  7.5e-07  2.6e-08  1.02e+00   -4.712013894e-01  -4.215515849e-01  1.8e-10  0.01  


...

MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3686) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3687) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3688) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3689) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3690) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3691) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3692) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3693) of matrix 'A'.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3694            
  Cones               

MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3686) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3687) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3688) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3689) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3690) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3691) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3692) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3693) of matrix 'A'.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3694            
  Cones               

MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3686) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3687) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3688) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3689) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3690) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3691) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3692) of matrix 'A'.
MOSEK warning 705: #8 (nearly) zero elements are specified in sparse row ''(3693) of matrix 'A'.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3694            
  Cones               

Excessive output truncated after 524385 bytes.

9   3.1e-08  3.7e-06  7.3e-07  1.53e+00   -2.855491775e-01  -2.794469574e-01  1.7e-09  10.28 
Optimizer terminated. Time: 10.51   


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: -2.8554917751e-01   nrm: 2e+03    Viol.  con: 3e-01    var: 0e+00    barvar: 0e+00  


In [303]:
plot(scatter(Nvec',log.(Tvec'),legend=false),scatter(Nvec',log.(Evec'),legend=false), layout=@layout([a b]))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 -20 
 
 
 -15 
 
 
 -10 
 
 
 -5

In [258]:
Ag = convert(Array,VectorOfArray(coefficients.(-differentiate(U6,x))))';
Ac = convert(Array,VectorOfArray(coefficients.(f6 + differentiate(U6,x))))';
Av = convert(Array,VectorOfArray(coefficients.(-differentiate(V6,x))))';
eig(Ac)[1]
NormalSoS.checknorm(f6,V6,x)

0.1464135511155301

In [293]:
n = 4;
@polyvar x[1:n]
A = randsys(n)
F(X::Vector) = A*X;
f = F(x);
U = NormalSoS.normdecomp(f,x, MosekSolver(),10)

Computed matrix A
Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^2, x1x2, x1x3, x1x4, x2^2, x2x3, x2x4, x3^2, x3x4, x4^2, x1, x2, x3, x4, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 634             
  Cones                  : 0               
  Scalar variables       : 19              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense       

0.23551973882602245x1^2 + 0.15295189680627733x1x2 + 0.050381157534484945x1x3 + 0.20403595510390007x1x4 + 0.1413563388643137x2^2 - 0.06207868194840708x2x3 + 0.014833989053307062x2x4 + 0.049264136997678724x3^2 + 0.06736713201945856x3x4 + 0.09615109361158525x4^2 + 1.1664588092866595

In [291]:
U = 2.0x[1]^2 + 3.4x[3] + NaN*x[2]
any(isnan.(coefficients(U)))


true